In [1]:
# Takes in the PhysVolName string (volline) and Returns the indices SAno, DUno
# Index Extractor Function
def indext(volline):
    DAind = volline.index("DetectorArray_") + len("DetectorArray_")
    DAno = int(volline[DAind :DAind + 3])
    SAind = volline.index("StringAssembly_") + len("StringAssembly_")
    SAno = int(volline[SAind:SAind + 3])
    DUind = volline.index("DetUnit_") + len("DetUnit_")
    DUno = int(volline[DUind:DUind + 3])
    ADind = volline.index("ActiveDet_") + len("ActiveDet_")
    ADno = int(volline[ADind:ADind + 3])
    return SAno, DUno

In [2]:
import uproot
import numpy
import pandas as pd
# Reading the input file
inpath = '/global/cfs/projectdirs/m2676/sims/LGND60_AsBuilt_Calibration/'+ 'LGND_200_60kg_Calibration_wSourceNumber_41_Height_-1000mm.root'
file = uproot.open(inpath)

In [3]:
# Examining the rootfile
file["fTree"].values()

[<TBranchElement 'fMCRun' (2 subbranches) at 0x7fffba7653a0>,
 <TBranchElement 'MGTDataObject' at 0x7fffba7658e0>,
 <TBranchElement 'MGMCRun' (30 subbranches) at 0x7fffba6f32b0>,
 <TBranchElement 'fRunID' at 0x7fffba6f37c0>,
 <TBranchElement 'fNEvents' at 0x7fffba6f9190>,
 <TBranchElement 'fDescription' at 0x7fffba6f9af0>,
 <TBranchElement 'fSensitiveVolumeIDs' at 0x7fffba701490>,
 <TBranchElement 'fWriteRandGenState' at 0x7fffba701d90>,
 <TBranchElement 'fWriteAllSteps' at 0x7fffba706730>,
 <TBranchElement 'fWriteAllStepsInEventsThatDepositEnergy' at 0x7fffba70c0d0>,
 <TBranchElement 'fWriteEventsThatDepositEnergyInGe' at 0x7fffba70ca30>,
 <TBranchElement 'fKillAlphas' at 0x7fffba7133d0>,
 <TBranchElement 'fKillBetas' at 0x7fffba713d30>,
 <TBranchElement 'fKillGammas' at 0x7fffba7196d0>,
 <TBranchElement 'fKillNeutrons' at 0x7fffba71e070>,
 <TBranchElement 'fStopNuclei' at 0x7fffba71e9d0>,
 <TBranchElement 'fUseTimeWindow' at 0x7fffba723370>,
 <TBranchElement 'fTimeWindow' at 0x7fffba

In [4]:
# Each event contains a series of energy deposited in each step
Edep=file["fTree"]["fSteps.fEdep"].array(library="numpy")
#print(Edep)

In [5]:
# The associated active volume in which the energy is deposited
PhysVolName=file["fTree"]["fSteps.fPhysVolName"].array(library="numpy")
#print(PhysVolName)

In [6]:
# The cood [i,j,k] : i = the row number is the evtid; j= det unit; k= string assembly
# No. of Strings = 12, No of Det per string = 8
#
# 'etab' is the table of energy deposited
etab=numpy.zeros([PhysVolName.size,8,12], dtype=object)# index runs from 0 to 196378
#print(etab)
#
# 'hits' is the table of hits in the region of interest )(defined below)
hits=numpy.zeros([8,12], dtype=object)# index runs from 0 to 196378
#print(hits)
#
# Defining the energy ROI, by defining the centroid and half-window
centE = 2.6145
deltaE = 0.0001
minE = centE - deltaE
maxE = centE + deltaE

In [7]:
# Looping over the events, extracting the energy deposited detector by detector and appropriately summing
# This takes care of possible scatters between detectors in a single event
for i in range(len(Edep)):
    for (E,volline) in zip(Edep[i],PhysVolName[i]):
        SAno,DUno=indext(volline)
        etab[i, DUno-1, SAno-1]=etab[i, DUno-1, SAno-1] + E

In [12]:
# Now that the total energy deposited in each event and in each detector has been computed,
# extracting the hits in the region of interest
for i in range(len(Edep)):
    for j in range(0,8):
        for k in range(0,12):
            if ((minE<= etab[i,j,k]) & (etab[i,j,k] <= maxE)):
                hits[j, k] = hits[j, k] + 1

In [24]:
print(etab[40])

[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0.5714850357227821 0 0 0 0 0 0 0 0 0 0 0]
 [0 0.12559136504641755 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]


In [9]:
print(hits)

[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 3 3 0 0 0 0]
 [1 3 0 0 0 0 7 3 0 0 0 0]
 [10 4 0 0 0 0 8 0 0 0 0 0]
 [15 32 0 0 0 0 18 0 0 0 0 0]
 [104 0 0 0 0 0 29 0 0 0 0 0]]


In [10]:
# Setting the output file
start = inpath.find('LGND_200')
end = inpath.find('.root')
outpath="./fresults/" + inpath[start:end] + ".txt"
print(outpath)

./fresults/LGND_200_60kg_Calibration_wSourceNumber_41_Height_-1000mm.txt


In [11]:
outfile = open(outpath,'w')
outfile.write("String assembly on the columns and detector units on the rows\n")
outfile.write("infile: ")
#outfile.write(infile)
outfile.write("\n")
numpy.savetxt(outfile, hits, fmt = "%4.0f") 
#numpy.savetxt(outfile, hits, fmt = "%6.0f") 
outfile.close()